# ハードサブ字幕処理ツール

このノートブックは、動画に焼き込まれたハードサブ（日英字幕）を抽出し、Anki学習カードを生成するためのツールです。

## 主な機能
- VideoSubFinderで抽出した字幕画像を処理
- 日本語と英語の字幕を分離・認識
- タイムスタンプ付きSRTファイルの生成
- 音声セグメントと画像を組み合わせたAnkiカードCSVの作成

## 使用シーン
日英字幕付きの動画から、語学学習用のAnkiフラッシュカードを自動生成する際に使用します。


In [ ]:

import os
import re
import csv
from moviepy import AudioFileClip

# ファイル名から開始時刻と終了時刻を抽出する関数
def extract_times(filename):
    match = re.match(r'(\d+_\d+_\d+_\d+)__(\d+_\d+_\d+_\d+)', filename)
    if match:
        start_time = match.group(1)
        end_time = match.group(2)
        return start_time, end_time
    return None, None

# 時刻形式を '0_01_07_123' から '00:01:07.123' に変換する関数
def convert_time_format(time_str):
    parts = time_str.split('_')
    return f"{int(parts[0]):02}:{int(parts[1]):02}:{int(parts[2]):02}.{parts[3]}"

# パス
image_input_folder = r'/path/to/VideoSubFinder/RGBImages'
image_output_folder = r'/path/to/output/segmentsImage'
audio_input_path = r'/path/to/your/video_audio.mp3'
audio_output_folder = r'/path/to/output/segmentsMP3'
csv_output_path = r'/path/to/output/anki_cards.csv'
prefix = 'card'

# 出力ディレクトリが存在しない場合は作成
os.makedirs(image_output_folder, exist_ok=True)
os.makedirs(audio_output_folder, exist_ok=True)

# CSVファイルを書き込み用に開く
with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Image', 'Audio'])

    # 入力フォルダ内の各画像ファイルを処理
    for filename in os.listdir(image_input_folder):
        if filename.endswith('.jpeg') or filename.endswith('.jpg'):
            start_time, end_time = extract_times(filename)
            if start_time and end_time:
                # 時刻形式を変換
                start_time_formatted = convert_time_format(start_time)
                end_time_formatted = convert_time_format(end_time)

                # 音声セグメントを切り出す
                audio_clip = AudioFileClip(audio_input_path).subclipped(start_time_formatted, end_time_formatted)
                audio_output_filename = f"{prefix}_{start_time}__{end_time}.mp3"
                audio_output_path = os.path.join(audio_output_folder, audio_output_filename)
                audio_clip.write_audiofile(audio_output_path, codec='mp3')

                # 画像ファイルをコピーして名前を変更
                image_output_filename = f"{prefix}_{start_time}__{end_time}.jpeg"
                image_output_path = os.path.join(image_output_folder, image_output_filename)
                os.rename(os.path.join(image_input_folder, filename), image_output_path)

                # CSVに書き込む
                csv_writer.writerow([f"<img src='{image_output_filename}'>", f"[sound:{audio_output_filename}]"])

print("AnkiカードCSVファイルの生成が完了しました。")


ハードサブの処理手順、srtファイルを取得する。

0.まず以下をクリアする
    /path/to/VideoSubFinder/RGBImages
    /path/to/VideoSubFinder/TXTImages、
    /path/to/VideoSubFinder/TXTResultsJP、
    /path/to/VideoSubFinder/TXTResultsEN。

1.VideoSubFinderWXWで動画から画像を取得、高さは日英文が同じ高さになるように調整。
    ソフトウェア操作。
    まず既存の画像をクリアする。

2.画像の二値化を取得。/path/to/VideoSubFinder/TXTImages。
    ソフトウェア操作。

3.画像を日英の2つの画像に分割する。
    コードで操作。
    /path/to/VideoSubFinder/cutENPictrue、/path/to/VideoSubFinder/cutJPPictrueに保存。

4.AABBYで上記2つのフォルダ内の画像中の文字をそれぞれ認識する。
    ソフトウェア操作。
    /path/to/VideoSubFinder/TXTResultsJP、/path/to/VideoSubFinder/TXTResultsENに保存。

5.上記フォルダ内の日英文の文書内の文字を結合する。
    コード操作。
    /path/to/VideoSubFinder/TXTResultsに保存。

6.VideoSubFinderWXWでsrt字幕文書を生成する。
    ソフトウェア操作。
    /path/to/output/subtitle.srtに保存。

7.字幕ファイルから空白の字幕を削除する。
    コード操作。
    /path/to/output/subtitle_cleaned.srtに保存。


In [ ]:
#3. 画像を日英それぞれの画像に切り分ける
import os
from PIL import Image

# 元の画像パス
src_dir = r'/path/to/VideoSubFinder/TXTImages'
# 保存先のパス
top_dir = r'/path/to/VideoSubFinder/cutENPictrue'
bottom_dir = r'/path/to/VideoSubFinder/cutJPPictrue'

# 出力フォルダを作成し、中の.jpegをクリアする
def prepare_dir(directory):
    os.makedirs(directory, exist_ok=True)
    for filename in os.listdir(directory):
        if filename.lower().endswith('.jpeg'):
            os.remove(os.path.join(directory, filename))

prepare_dir(top_dir)
prepare_dir(bottom_dir)

# すべてのJPEG画像をスキャン
for filename in os.listdir(src_dir):
    if filename.lower().endswith('.jpeg'):
        img_path = os.path.join(src_dir, filename)
        with Image.open(img_path) as img:
            width, height = img.size
            split_line = int(height * 0.6)

            # 画像を分割
            top_img = img.crop((0, 0, width, split_line))
            bottom_img = img.crop((0, split_line, width, height))

            # 画像を保存
            top_img.save(os.path.join(top_dir, filename))
            bottom_img.save(os.path.join(bottom_dir, filename))

print("画像のクリアと分割が完了しました！")


In [ ]:
#5. 上記フォルダ内の日英テキストを結合
import os

# フォルダパス
jp_dir = r'/path/to/VideoSubFinder/TXTResultsJP'
en_dir = r'/path/to/VideoSubFinder/TXTResultsEN'
out_dir = r'/path/to/VideoSubFinder/TXTResults'

# 出力フォルダ内の.txtを再作成してクリア
os.makedirs(out_dir, exist_ok=True)
for filename in os.listdir(out_dir):
    if filename.lower().endswith('.txt'):
        os.remove(os.path.join(out_dir, filename))

# 日本語ディレクトリをスキャン
for filename in os.listdir(jp_dir):
    if filename.lower().endswith('.txt'):
        jp_path = os.path.join(jp_dir, filename)
        en_path = os.path.join(en_dir, filename)
        out_path = os.path.join(out_dir, filename)

        try:
            # 日本語ファイルの先頭行を読む
            with open(jp_path, 'r', encoding='utf-8') as jp_file:
                jp_line = jp_file.readline().strip()

            # 英語ファイルの先頭行を読む
            with open(en_path, 'r', encoding='utf-8') as en_file:
                en_line = en_file.readline().strip()

            # 新しいファイルに書き込む
            with open(out_path, 'w', encoding='utf-8') as out_file:
                out_file.write(jp_line + '\n' + en_line + '\n')

        except FileNotFoundError:
            print(f"対応する英語ファイルが見つかりません：{en_path}")
        except Exception as e:
            print(f"ファイル {filename} の処理中にエラーが発生しました：{e}")

print("TXTResults ディレクトリをクリアし、ファイルを再生成しました！")


In [ ]:
#7. 字幕ファイルから空白エントリを削除
import re

# SRTファイルのパス
srt_path = r'/path/to/your/subtitle.srt'

# 元の内容を読み込む
with open(srt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 整理後の字幕ブロックを保持
cleaned_blocks = []
block = []

# ブロックごとに処理
for line in lines:
    if line.strip() == "":
        if block:
            # 番号とタイムコードだけで本文が無いか判定
            if len(block) < 3 or all(line.strip() == '' for line in block[2:]):
                pass  # 無効ブロックはスキップ（削除）
            else:
                cleaned_blocks.extend(block + [''])  # 正常なブロックを残す
            block = []
    else:
        block.append(line.rstrip('\n'))

# 末尾に空行が無い場合を処理
if block:
    if len(block) >= 3 and not all(line.strip() == '' for line in block[2:]):
        cleaned_blocks.extend(block + [''])

# クリーンアップ後の内容で元ファイルを上書き
with open(srt_path, 'w', encoding='utf-8') as file:
    file.write('\n'.join(cleaned_blocks).strip() + '\n')

print("空の字幕ブロックのクリアが完了しました！")
